In [58]:
from src.helper import export_projects, dataset_metadata, _export, download_datasets, upload_dataset, _export, import_projects, _convert_to_snake
import datarobot as dr
import shutil
import pandas as pd
import json

In [2]:
DIR = "test"
PROJECT_EXPORT_FILE = "new_projects.json"

In [3]:
projects = export_projects()
# catalog = get_datasets(projects)
catalog_metadata = dataset_metadata(projects)

In [1]:
catalog_metadata

In [5]:
results = []
for p in projects:
    md = _export(f"datasets/{p['catalogId']}/versions/?orderBy=created")
    if "data" in md:
        results.extend(md["data"])
results_df = pd.DataFrame(results)
results_df = results_df.drop(columns=["datasetId", "versionId"])

In [2]:
results_df

In [3]:
next_batch = catalog_metadata[0:2] #loc[[0,1,5]]
next_batch["datasetSize(MB)"].sum()

In [4]:
raw = shutil.disk_usage(".")
free_mb = int(raw.free / (1000**2))
print("空き容量: ", free_mb , "MB")
print("次回実行分に必要な空き容量確保できる：：", next_batch["datasetSize(MB)"].sum() < free_mb)


In [5]:
new_catalog = download_datasets(next_batch, DIR, 4)
new_catalog

In [10]:
new_datasets = []
for cat in new_catalog:
    record = {"name": cat["name"]}
    if cat["complete"]:
        resp = upload_dataset(cat, DIR=DIR)
        if "catalogId" in resp:
            record = resp | {"name": cat["name"]}
        new_datasets.append(record)

In [12]:
for idx, p in enumerate(projects):
    matched_file = [ds for ds in new_datasets if ds["name"] == p["fileName"]][0]
    projects[idx]["catalogId"] = matched_file["catalogId"]

In [13]:
with open(f"{DIR}/{PROJECT_EXPORT_FILE}", "w") as f:
    json.dump(projects, f, ensure_ascii=False, indent=4)

In [ ]:
print("データセット登録は時間がかかる場合があります。ファイルサイズによっては数秒〜数分待ってかれプロジェクトのマイグレを始めてください。")

In [30]:
#Given current spec, metadata is ONLY for export (source environment), sigh...
new_catalog = dataset_metadata(projects,env="TARGET")

In [6]:
new_catalog

In [ ]:
new_projects = []
mode_map = {1:"autopilot",2:"manual",3:"quick", 4:"comprehensive"}
adv_option_filter = ["downsampledMinorityRows", "downsampledMajorityRows","responseCap"]
for proj in projects:
    print("Creating project: ", proj["projectName"])
    project = dr.Project.create_from_dataset(
        dataset_id=proj["catalogId"],
        project_name=proj["projectName"],
    )
    #TypeError: AdvancedOptions.__init__()のエラーがスローされたら該当属性を以下の配列に追加する
    adv_options = {k:v for (k,v) in proj["advancedOptions"].items() if k not in adv_option_filter}
    adv_options = _convert_to_snake(adv_options)
    advanced_options = dr.AdvancedOptions(**adv_options)
    new_proj = project.analyze_and_model(
        #supervised / 教師あり前提でのサンプルコード
        #unsupervised_mode: True
        #上記の場合はtargetあってはならない
        #unsupervised_type = ??
                    target=  proj["target"],
                    mode=    mode_map[proj["autopilotMode"]],
                    target_type=  proj["targetType"],
                    metric = proj["metric"],
                    positive_class = proj["positiveClass"],
                    advanced_options=advanced_options

        #未確認アイテム 
        #partition = proj["partition"],
        
        #holdout_unlocked = proj["holdoutUnlocked"],
        #stage = proj["stage"],
        #max_train_pct = proj["maxTrainPct"],
    )
    new_projects.append(new_proj)